In [ ]:

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download -d ananthu017/emotion-detection-fer


Dataset URL: https://www.kaggle.com/datasets/ananthu017/emotion-detection-fer
License(s): CC0-1.0


In [ ]:
import zipfile
with zipfile.ZipFile('/content/emotion-detection-fer.zip', 'r') as zip_ref:
    zip_ref.extractall('/content')

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt

In [ ]:
image_size = (160, 160)

train_ds = keras.utils.image_dataset_from_directory(
    '/content/train',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=image_size,
    shuffle=True
)

val_ds = keras.utils.image_dataset_from_directory(
    '/content/test',
    labels='inferred',
    label_mode='int',
    batch_size=16,
    image_size=image_size
)

AUTOTUNE = tf.data.AUTOTUNE

def preprocess(image, label):
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    return image, label

train_ds = train_ds.map(preprocess).shuffle(500).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.map(preprocess).cache().prefetch(buffer_size=AUTOTUNE)

Found 28709 files belonging to 7 classes.
Found 7178 files belonging to 7 classes.


In [ ]:
# 5. Build the transfer learning model

base_model = MobileNetV2(
    input_shape=(160, 160, 3),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = True  # Freeze base model

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# 6. Compile model
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(7, activation='softmax')
])

In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# 7. Callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_mobilenet_model.h5', save_best_only=True, monitor='val_loss', mode='min')

In [ ]:
# 8. Train the model

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=15,
    callbacks=[early_stop, checkpoint]
)

Epoch 1/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 71s 37ms/step - accuracy: 0.6878 - loss: 0.8599 - val_accuracy: 0.5952 - val_loss: 1.2055
Epoch 2/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 70s 37ms/step - accuracy: 0.7022 - loss: 0.8143 - val_accuracy: 0.5715 - val_loss: 1.2865
Epoch 3/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 82s 37ms/step - accuracy: 0.7197 - loss: 0.7885 - val_accuracy: 0.6163 - val_loss: 1.2321
Epoch 4/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 71s 38ms/step - accuracy: 0.7303 - loss: 0.7423 - val_accuracy: 0.6115 - val_loss: 1.1874
Epoch 5/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 81s 38ms/step - accuracy: 0.7453 - loss: 0.7020 - val_accuracy: 0.6096 - val_loss: 1.2214
Epoch 6/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 71s 37ms/step - accuracy: 0.7666 - loss: 0.6625 - val_accuracy: 0.6116 - val_loss: 1.1997
Epoch 7/15
1795/1795 ━━━━━━━━━━━━━━━━━━━━ 82s 38ms/step - accuracy: 0.7770 - loss: 0.6291 - val_accuracy: 0.6290 - val_loss: 1.2351


In [ ]:
# 9. Plot training accuracy

plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('MobileNetV2 Accuracy')
plt.legend()
plt.show()

In [ ]:
# 10. Final evaluation

model.evaluate(val_ds)